<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/Inflation_swap_ZC_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling Inflation swaps using QuantLib

<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/Inflation_swap_ZC_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
calculation_date = ql.Date(20, 10, 2015)
ql.Settings.instance().evaluationDate = calculation_date

Okay, here is a Python Jupyter Notebook example demonstrating how to price both a Zero-Coupon Inflation Swap (ZCIS) and a Year-on-Year Inflation Swap (YoYIS) using the QuantLib library.

**Assumptions:**

*   You have QuantLib-Python installed (`pip install QuantLib-Python`).
*   The market data (rates, fixings) used here is purely **illustrative** and not real market data.
*   We will bootstrap a simple inflation curve from ZCIS quotes.

In [ ]:
# Pricing Inflation Swaps with QuantLib

# This notebook demonstrates how to set up the necessary components and price:
# 1. A Zero-Coupon Inflation Swap (ZCIS)
# 2. A Year-on-Year Inflation Swap (YoYIS)
# using the QuantLib library in Python.

# ## 1. Imports and Setup

import QuantLib as ql
import datetime

# ## 2. Evaluation Date and Market Data Setup

# Set the evaluation date
eval_date = ql.Date(15, ql.May, 2023)
ql.Settings.instance().evaluationDate = eval_date

# Define calendar and conventions
calendar = ql.UnitedKingdom() # Example calendar
business_convention = ql.ModifiedFollowing
day_counter_fixed = ql.Actual365Fixed() # Day counter often used for ZCIS/Inflation Legs
day_counter_discount = ql.Actual365Fixed()

# ---
# ### 2.1 Discount Curve (Nominal Yield Curve)
# For simplicity, we'll use a flat forward curve. In practice, you'd bootstrap this from market instruments (deposits, futures, swaps).

discount_rate = ql.SimpleQuote(0.05) # 5% flat rate
discount_curve = ql.FlatForward(eval_date, ql.QuoteHandle(discount_rate), day_counter_discount)
discount_curve_handle = ql.YieldTermStructureHandle(discount_curve)

print("Discount Curve Setup Complete.")

Discount Curve Setup Complete.


In [ ]:
# ---
# ### 2.2 Inflation Index Definition
# We need to define the specific inflation index (e.g., UK RPI, US CPI).

region = ql.UnitedKingdomRegion()
frequency = ql.Monthly # Frequency of index publication
index_interpolated = True # Is the index level interpolated between fixing dates? (Commonly True for swaps)
observation_lag = ql.Period(2, ql.Months) # Lag between reference period and publication (e.g., 2-3 months)

# Create the Zero Coupon Inflation Index object
# False = historical fixings are NOT available adjusted. True = available adjusted
uk_rpi_zero = ql.UKRPI(index_interpolated)

# Add some *illustrative* historical fixings
# Fixings are needed for projecting inflation near the evaluation date, especially with interpolation
# Format: index.addFixing(Date, value)
# These dates MUST correspond to the index publication schedule (e.g., mid-month for UKRPI)
# Using realistic dates relative to eval_date and observation_lag
# Need fixings for March and April to interpolate May value based on 2M lag
uk_rpi_zero.addFixing(ql.Date(15, ql.March, 2023), 300.0) # Hypothetical value
uk_rpi_zero.addFixing(ql.Date(15, ql.April, 2023), 301.5) # Hypothetical value
uk_rpi_zero.addFixing(ql.Date(15, ql.May, 2023), 302.8) # Hypothetical value (needed if lag=0/1M)

print(f"Inflation Index ({uk_rpi_zero.name()}) Setup Complete.")
print(f"Observation Lag: {observation_lag}")

In [ ]:
# ---
# ### 2.3 Inflation Term Structure (Bootstrapping from ZCIS Quotes)
# We bootstrap the inflation curve from market quotes for Zero-Coupon Inflation Swaps.

# Illustrative Market Data for ZCIS (Rate = Zero Coupon Inflation Rate)
zc_swap_maturities = [ql.Period(y, ql.Years) for y in [1, 2, 3, 5, 7, 10]]
# These rates are the *inflation* rates, not nominal rates
zc_swap_rates = [0.035, 0.036, 0.037, 0.038, 0.039, 0.040] # Example % inflation per year

# Create Rate Helpers
zc_helpers = []
for i, mat in enumerate(zc_swap_maturities):
    rate = ql.QuoteHandle(ql.SimpleQuote(zc_swap_rates[i]))
    maturity_date = calendar.advance(eval_date, mat)
    helper = ql.ZeroCouponInflationSwapHelper(
        rate,
        observation_lag,
        maturity_date,
        calendar,
        business_convention,
        day_counter_fixed,
        uk_rpi_zero, # The zero coupon index
        discount_curve_handle # Nominal curve for discounting ZCIS cashflows during bootstrap
    )
    zc_helpers.append(helper)

# Bootstrap the Inflation Curve (Piecewise Zero-Coupon Inflation Curve)
# Note: Need to specify interpolation method on the *zero inflation rates*
# Common methods: LogLinear, Linear
base_zero_rate = zc_swap_rates[0] # Base rate for bootstrapping start
inflation_curve = ql.PiecewiseZeroInflationCurve_LogLinear( # Or PiecewiseZeroInflationCurve_Linear
    eval_date,
    calendar,
    day_counter_fixed,
    observation_lag,
    frequency,
    index_interpolated,
    discount_curve_handle, # Pass nominal curve handle again
    zc_helpers
)
inflation_curve.enableExtrapolation()
inflation_curve_handle = ql.ZeroInflationTermStructureHandle(inflation_curve)

print("Inflation Curve Bootstrapping Complete.")

In [ ]:
# Check a bootstrapped zero inflation rate
test_date = calendar.advance(eval_date, ql.Period(5, ql.Years))
print(f"Bootstrapped Zero Inflation Rate at {test_date}: {inflation_curve.zeroRate(test_date, observation_lag, True):.4%}")


# ## 3. Defining the Inflation Swaps

# Swap General Parameters
notional = 1000000.0
swap_start_date = eval_date # Or a future settlement date
swap_maturity_years = 5
swap_maturity_date = calendar.advance(swap_start_date, ql.Period(swap_maturity_years, ql.Years))

print(f"\n--- Swap Parameters ---")
print(f"Notional: {notional:,.0f}")
print(f"Start Date: {swap_start_date}")
print(f"Maturity Date: {swap_maturity_date}")

In [ ]:
# ---
# ### 3.1 Zero-Coupon Inflation Swap (ZCIS) Definition

# ZCIS pays: Notional * [(Index_End / Index_Start) - 1] on the inflation leg
# ZCIS pays: Notional * [(1 + FixedRate)^Years - 1] on the fixed leg (or equivalent single payment)
# QuantLib handles the exact calculation based on day count.

# We define the swap assuming *we* pay the fixed rate (Payer swap)
fixed_rate_zcis = 0.038 # The fixed rate agreed in the contract
swap_type_zcis = ql.ZeroCouponInflationSwap.Payer # Payer = Pay Fixed, Receive Inflation

zcis = ql.ZeroCouponInflationSwap(
    swap_type_zcis,
    notional,
    swap_start_date,
    swap_maturity_date,
    calendar,
    business_convention,
    day_counter_fixed,
    fixed_rate_zcis,       # The fixed rate of the swap contract
    uk_rpi_zero,           # The ZC Inflation Index
    observation_lag,
    # Optional: Specify interpolation on index observation explicitly (default is index setting)
    # ql.CPI.AsIndex
)

print("\nZero-Coupon Inflation Swap (ZCIS) Defined.")
print(f"Fixed Rate: {fixed_rate_zcis:.4%}")
print(f"Type: {'Payer (Pay Fixed)' if swap_type_zcis == ql.ZeroCouponInflationSwap.Payer else 'Receiver (Receive Fixed)'}")

In [ ]:
# ---
# ### 3.2 Year-on-Year Inflation Swap (YoYIS) Definition

# YoYIS pays periodic coupons based on the Year-on-Year inflation rate vs a fixed rate.

# Need a YoY Inflation Index linked to the bootstrapped curve
# This index forecasts future YoY rates using the inflation curve
yoy_inflation_index = ql.YoYInflationIndex(
    frequency,             # Match the frequency of the underlying index ? No, this is YoY index freq
    region,                # Match region
    False,                 # Availability of revised fixings (usually False)
    index_interpolated,    # Match interpolation
    False,                 # Use ratio of indices or difference? Usually False (use ratio)
    inflation_curve_handle # Link to the bootstrapped inflation curve for forecasting
    # Frequency argument here is confusing, often Monthly for underlying, but YoY swap can be Annual/SemiAnnual
    # Let's try ql.Annual frequency for the YoY index itself
    # ql.Monthly # Try matching underlying frequency first
)
# Correction: The YoYInflationIndex needs the *ZeroInflationIndex* as input, not the curve directly in older QL versions
# Let's redefine using the more standard constructor pattern
yoy_inflation_index = ql.YoYInflationIndex(
    uk_rpi_zero.name(), # Pass underlying index name or family name
    region,
    False, # revised fixings
    index_interpolated, # interpolated?
    True, # Use Ratio? (I[t]/I[t-1Y] - 1) -> Usually True for YoY Swaps
    frequency, # Underlying Index Freq
    observation_lag,
    inflation_curve_handle # Inflation curve handle for forecasting
)

In [ ]:
# Swap Parameters for YoYIS
fixed_rate_yoy = 0.0375 # The fixed rate agreed in the contract
fixed_leg_day_counter = ql.Actual360() # Common day count for fixed legs
yoy_leg_day_counter = ql.Actual360() # Common day count for inflation legs
payment_frequency = ql.Period(1, ql.Years) # Annual payments

swap_type_yoy = ql.YearOnYearInflationSwap.Payer # Payer = Pay Fixed, Receive YoY Inflation

# Define Schedules for fixed and inflation legs
fixed_schedule = ql.MakeSchedule(
    swap_start_date,
    swap_maturity_date,
    payment_frequency,
    calendar=calendar,
    convention=business_convention,
    # Optional: rule=ql.DateGeneration.Forward (Default)
)

yoy_schedule = ql.MakeSchedule(
    swap_start_date,
    swap_maturity_date,
    payment_frequency,
    calendar=calendar,
    convention=business_convention,
    # Optional: rule=ql.DateGeneration.Forward (Default)
)


yoyis = ql.YearOnYearInflationSwap(
    swap_type_yoy,
    notional,
    fixed_schedule,         # Schedule for fixed leg payments
    fixed_rate_yoy,         # Agreed fixed rate
    fixed_leg_day_counter,  # Day counter for fixed leg accrual
    yoy_schedule,           # Schedule for YoY inflation leg payments
    yoy_inflation_index,    # The YoY Inflation Index object
    observation_lag,        # Observation lag for YoY readings
    yoy_leg_day_counter     # Day counter for YoY leg accrual
    # Optional: payment_lag, final_payment_lag etc.
)


print("\nYear-on-Year Inflation Swap (YoYIS) Defined.")
print(f"Fixed Rate: {fixed_rate_yoy:.4%}")
print(f"Payment Frequency: {payment_frequency}")
print(f"Type: {'Payer (Pay Fixed)' if swap_type_yoy == ql.YearOnYearInflationSwap.Payer else 'Receiver (Receive Fixed)'}")

In [ ]:
# ## 4. Pricing the Swaps

# Create the Pricing Engine
# For inflation swaps, we use the DiscountingSwapEngine, providing the nominal discount curve.
# The inflation curve is implicitly used via the Inflation Index linked to it.
pricing_engine = ql.DiscountingSwapEngine(discount_curve_handle)

# Set the engine for both swaps
zcis.setPricingEngine(pricing_engine)
yoyis.setPricingEngine(pricing_engine)

# ---
# ### 4.1 Calculate Net Present Value (NPV)

npv_zcis = zcis.NPV()
npv_yoy = yoyis.NPV()

print(f"\n--- Pricing Results ---")
print(f"ZCIS NPV: {npv_zcis:,.2f}")
print(f"YoYIS NPV: {npv_yoy:,.2f}")

In [ ]:
# Interpretation:
# Positive NPV for Payer means the fixed rate being paid is too low compared to market expectations (swap is valuable to payer).
# Negative NPV for Payer means the fixed rate being paid is too high compared to market expectations (swap has negative value to payer).

# ---
# ### 4.2 Calculate Fair Rate

# The fair rate is the fixed rate that makes the NPV of the swap equal to zero at inception.

fair_rate_zcis = zcis.fairRate()
fair_rate_yoy = yoyis.fairRate()

print(f"\nZCIS Fair Fixed Rate: {fair_rate_zcis:.4%}")
print(f"YoYIS Fair Fixed Rate: {fair_rate_yoy:.4%}")

# Comparison:
# If ZCIS Fixed Rate (0.0380) > Fair Rate -> Negative NPV for Payer
# If ZCIS Fixed Rate (0.0380) < Fair Rate -> Positive NPV for Payer
# (Matches our results if bootstrap was consistent with 5Y ZCIS rate)

# If YoYIS Fixed Rate (0.0375) > Fair Rate -> Negative NPV for Payer
# If YoYIS Fixed Rate (0.0375) < Fair Rate -> Positive NPV for Payer
# (Check if this matches the NPV result)

In [ ]:
# ## 5. Inspecting Cash Flows (Optional)

print("\n--- ZCIS Cash Flows (Illustrative) ---")
# ZCIS typically has one fixed and one inflation cashflow at maturity
# Accessing legs: leg(0) is usually fixed, leg(1) is usually inflation for Payer swap
try:
    fixed_leg_zcis = zcis.leg(0)
    inflation_leg_zcis = zcis.leg(1)

    # Note: ZCIS cashflows might be represented differently internally.
    # The structure is simple: one payment vs one payment at maturity.
    # A more detailed inspection might require looking at internal swap structure or specific methods.
    # Let's just print the expected payment dates from the structure
    print(f"Expected Fixed Payment Date (ZCIS): {fixed_leg_zcis[0].date()}") # Only one coupon
    print(f"Expected Inflation Payment Date (ZCIS): {inflation_leg_zcis[0].date()}") # Only one coupon
    # Actual amounts require deeper digging or running valuation logic manually
except Exception as e:
    print(f"Could not retrieve ZCIS leg details directly: {e}")


print("\n--- YoYIS Cash Flows ---")
fixed_leg_yoy = yoyis.leg(0) # Fixed Leg
yoy_leg_yoy = yoyis.leg(1)   # YoY Inflation Leg

print("Fixed Leg Coupons (YoYIS):")
for i, cf in enumerate(fixed_leg_yoy):
    coupon = ql.as_fixed_rate_coupon(cf)
    print(f"  Coupon {i+1}: Date={coupon.date()}, Accrual Start={coupon.accrualStartDate()}, Accrual End={coupon.accrualEndDate()}, Amount={coupon.amount():,.2f}, Rate={coupon.rate():.4%}")

print("\nYoY Inflation Leg Coupons (YoYIS):")
for i, cf in enumerate(yoy_leg_yoy):
    # These are InflationCoupons, specifically YoYInflationCoupons
    coupon = ql.as_yoy_inflation_coupon(cf)
    # The amount is calculated based on forecast YoY rate at valuation date
    print(f"  Coupon {i+1}: Date={coupon.date()}, Accrual Start={coupon.accrualStartDate()}, Accrual End={coupon.accrualEndDate()}, Amount={coupon.amount():,.2f}, Fixing Date={coupon.fixingDate()}, Index Rate={coupon.indexFixing():.4f} (Forecasted)")
    # Note: indexFixing() gives the *forecasted* YoY rate used for this coupon's calculation as of eval_date

print("\n\nNotebook Execution Finished.")

**Explanation:**

1.  **Imports and Setup:** Import `QuantLib` and `datetime`.
2.  **Evaluation Date & Market Setup:** Define the date for which the pricing is performed (`eval_date`). Set up basic calendar and conventions.
3.  **Discount Curve:** A nominal yield curve is needed to discount all cash flows to the present value. Here, a simple `FlatForward` curve is used for demonstration. In reality, this would be bootstrapped from market instruments (like OIS swaps for collateralized pricing).
4.  **Inflation Index:** Define the specific index (e.g., `UKRPI`). Key parameters are `observation_lag` (how long before the reference period end the fixing is published) and `interpolated` (whether index values are linearly interpolated between official fixing dates, common for swaps). Historical fixings around the evaluation date are crucial, especially for interpolation and handling the observation lag correctly.
5.  **Inflation Term Structure:** This curve represents the market's expectation of future inflation. It's bootstrapped here using `ZeroCouponInflationSwapHelper` objects created from market ZCIS quotes. The `PiecewiseZeroInflationCurve_LogLinear` class constructs the curve. This curve is essential for forecasting future inflation index levels or rates.
6.  **ZCIS Definition:** Create the `ZeroCouponInflationSwap` object, specifying whether you pay or receive the fixed rate (`Payer`/`Receiver`), notional, dates, the fixed rate, the inflation index, and the observation lag.
7.  **YoYIS Definition:**
    *   First, create a `YoYInflationIndex`. This special index uses the bootstrapped *inflation term structure* (`inflation_curve_handle`) to forecast future *year-on-year* inflation rates.
    *   Define payment schedules (`MakeSchedule`) for both the fixed and inflation legs.
    *   Create the `YearOnYearInflationSwap` object, specifying type, notional, schedules, fixed rate, day counters, the YoY index, and the observation lag.
8.  **Pricing Engine:** The `DiscountingSwapEngine` is used. It takes the *nominal discount curve* handle. The engine automatically picks up the correct inflation curve forecasting mechanism via the inflation index object attached to the swap.
9.  **Pricing:**
    *   `.setPricingEngine()` assigns the engine to the swap instruments.
    *   `.NPV()` calculates the Net Present Value.
    *   `.fairRate()` calculates the fixed rate that would make the swap's NPV zero.
10. **Cash Flows (Optional):** Demonstrates how to access the individual legs (`.leg(0)`, `.leg(1)`) and iterate through the cash flows (`ql.Coupon`, `ql.FixedRateCoupon`, `ql.YoYInflationCoupon`) to inspect payment dates, accrual periods, and calculated/forecasted amounts and rates.

This notebook provides a solid foundation for pricing inflation swaps in QuantLib. Remember to replace the illustrative market data with actual market quotes and build more robust curves for real-world applications.